<a href="https://colab.research.google.com/github/marianobonelli/Introduccion_a_Python_2024/blob/main/Introduccion_a_Python_2024_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la programación y análisis de datos con Python orientado a la producción agropecuaria

[![Cronograma](https://img.shields.io/badge/-Cronograma-blue?style=flat&logo=google-docs&logoColor=white)](https://docs.google.com/document/d/1RkiQrAyWAuJglmTdKCfa4QQ4JYHh0Klg6iRRzxSv8cc/edit?usp=sharing) [![Repositorio de GitHub](https://img.shields.io/badge/-Repositorio%20de%20GitHub-blue?style=flat&logo=github&logoColor=white)](https://github.com/marianobonelli/Introduccion_a_Python_2024)
<!-- [![Video de la clase](https://img.shields.io/badge/-Video%20de%20la%20clase-blue?style=flat&logo=youtube&logoColor=white)](https://youtu.be/eS30BzqMUR8) -->




**Clase 7: Jueves 12 de Septiembre**


Manejo y visualización de datos espaciales con geopandas y folium

---

# Mapas y geolocalización con geopandas y folium.

### Importamos archivos de prueba:

In [ ]:
# Descargar los archivos usando wget
!wget https://raw.githubusercontent.com/marianobonelli/Introduccion_a_Python_2024/main/assets/rindes_lotes_9_10.zip -O rindes_lotes_9_10.zip
!wget https://raw.githubusercontent.com/marianobonelli/Introduccion_a_Python_2024/main/assets/lotes.zip -O lotes.zip

## Geopandas

[<img src="https://geopandas.org/en/stable/_static/geopandas_logo_web.svg" width=200px>](https://geopandas.org/en/stable/docs.html)



GeoPandas, como su nombre sugiere, extiende la popular biblioteca de ciencia de datos pandas añadiendo soporte para datos geoespaciales. Si no estás familiarizado con pandas, te recomendamos echar un vistazo rápido a su documentación de inicio antes de proceder.

La estructura de datos principal en GeoPandas es el geopandas.GeoDataFrame, una subclase de pandas.DataFrame, que puede almacenar columnas de geometría y realizar operaciones espaciales. El geopandas.GeoSeries, una subclase de pandas.Series, maneja las geometrías. Por lo tanto, tu GeoDataFrame es una combinación de pandas.Series, con datos tradicionales (numéricos, booleanos, texto, etc.), y geopandas.GeoSeries, con geometrías (puntos, polígonos, etc.). Puedes tener tantas columnas con geometrías como desees; no hay límite típico para el software GIS de escritorio.

[<img src="https://geopandas.org/en/stable/_images/dataframe.svg" width=650px>](https://geopandas.org/en/stable/getting_started/introduction.html)

 ### Lectura de archivos como GeoDataFrame (GDF)

 [geopandas.read_file()](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)

In [ ]:
import geopandas as gpd

lotes_gdf = gpd.read_file('lotes.zip')
lotes_gdf.head()

In [ ]:
lotes_gdf.info()

In [ ]:
# Uso de try para manejo de errores

try:
  lotes_gdf = gpd.read_file('lotes.zipp')
  lotes_gdf.head()
except Exception as e:
  print(f'Ocurrió un error: {e}')

In [ ]:
rindes_gdf = gpd.read_file('rindes_lotes_9_10.zip')
rindes_gdf.head()

#### Uso de pyogrio para acelerar la lectura de archivos

In [ ]:
!pip install pyogrio

In [ ]:
rindes_gdf = gpd.read_file('rindes_lotes_9_10.zip', engine="pyogrio")
rindes_gdf.head()

In [ ]:
rindes_gdf.info()

### Visualización de los GDF

#### Mapas estáticos nativos de geopandas

[Making maps and plots](https://geopandas.org/en/stable/docs/user_guide/mapping.html)

In [ ]:
# Visualización de los gdf

rindes_gdf.plot(column="Yld_Mass_D",
                cmap='RdYlGn', # escala de colores: https://matplotlib.org/stable/users/explain/colors/colormaps.html
                markersize=3 # tamaño de los puntos
                )

##### Visualización por cuantiles

In [ ]:
import pandas as pd

# Primero, crearemos categorías basadas en cuantiles para la columna 'Yld_Mass_D'
rindes_gdf['cuantiles'] = pd.qcut(rindes_gdf['Yld_Mass_D'], 4, labels=False)

# Graficamos ahora la nueva columna cuantiles
rindes_gdf.plot(column='cuantiles',
                cmap='RdYlGn', # escala de colores: https://matplotlib.org/stable/users/explain/colors/colormaps.html
                markersize=3 # tamaño de los puntos
                )

#### Mapas interactivos de geopandas con folium

GeoPandas ofrece una amplia variedad de opciones para visualizar datos espaciales. Entre ellas, destaca el método .explore, que utiliza Folium para crear mapas interactivos.

Puedes acceder a la documentación de GeoPandas sobre mapeo interactivo [aquí](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html).

Si deseas obtener más información sobre Folium y la creación de mapas interactivos, consulta su documentación [aquí](https://python-visualization.github.io/folium/latest/getting_started.html).

In [ ]:
# Es necesario instalar algunas librerías extras primero

!pip install folium matplotlib mapclassify

In [ ]:
import folium

# Crear el mapa
mapa = lotes_gdf.explore(
    column="Cultivo",  # hacer un coropleta basado en la columna "Cultivo"
    tooltip=["Lote", 'Cultivo', 'Hibrido/Va',  'Surface (h'],  # mostrar valor en tooltip (al pasar el mouse)
    popup=True,  # mostrar todos los valores en popup (al hacer clic)
    tiles="OpenStreetMap",  # "CartoDB positron" / "OpenStreetMap" / "Esri.WorldImagery"
    cmap="Set1",  # usar el mapa de colores "Set1" de matplotlib # cmap = https://matplotlib.org/stable/users/explain/colors/colormaps.html
    style_kwds=dict(color="black"),  # usar contorno negro
    name="Lotes",  # nombre del mapa
)

folium.LayerControl().add_to(mapa)  # agregar control de capas

# Guardar el mapa como un archivo HTML
mapa.save("mapa_lotes.html")

mapa

#### Mapas con matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

# Utilizar una columna del GeoDataFrame para los colores. Reemplaza 'tu_columna_color' con el nombre de tu columna
lotes_gdf.plot(ax=ax,
               column='Cultivo', # columna a partir de la cual se asigna simbología
               cmap="Set1", # cmap = https://matplotlib.org/stable/users/explain/colors/colormaps.html
               edgecolor='black', # contorno negro
               legend=False, # sin leyenda
               )

# Graficar los puntos
rindes_gdf.plot(ax=ax,
                column='cuantiles', # columna a partir de la cual se asigna simbología
                cmap="RdYlGn", # cmap = https://matplotlib.org/stable/users/explain/colors/colormaps.html
                markersize=0.2, # tamaño de los puntos
                )

# Quitar los ejes
ax.set_axis_off()

plt.show()

#### Mapas con Plotly

In [ ]:
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth_mapbox.html

import plotly.express as px

# Crear figura para los lotes
fig = px.choropleth_mapbox(
    lotes_gdf,  # GeoDataFrame que se desea graficar
    geojson=lotes_gdf.geometry.__geo_interface__,  # Convierte las geometrías del GeoDataFrame a formato GeoJSON para que Plotly pueda procesarlas
    locations=lotes_gdf.index,  # Utiliza los índices del GeoDataFrame para asignar las geometrías a cada fila
    color='Cultivo',  # Usar la columna 'Cultivo' para la simbología
)

# Calcular el centroide del conjunto de polígonos
centroid = lotes_gdf.geometry.unary_union.centroid

# Actualizar el layout para centrar el mapa
fig.update_layout(
    mapbox_style="open-street-map",  # Establece el estilo del mapa base, utilizando el estilo de "OpenStreetMap"
    mapbox_zoom=12,  # Define el nivel de zoom del mapa; 12 es un valor intermedio que muestra detalles sin acercarse demasiado
    mapbox_center={"lat": centroid.y, "lon": centroid.x},  # Centra el mapa en el centroide calculado (latitud y longitud)
    # margin={"r": 0, "t": 0, "l": 0, "b": 0},  # Elimina los márgenes alrededor del gráfico para maximizar el área de visualización
)

fig.show()

### Algunas funcionalidades de geopandas

* [Proyecciones automáticas](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.estimate_utm_crs.html#geopandas.GeoDataFrame.estimate_utm_crs)

* [Overlay](https://geopandas.org/en/stable/docs/user_guide/set_operations.html)

* [Muchos otros](https://geopandas.org/en/stable/docs/reference/geoseries.html)

In [ ]:
# Estimar CRS UTM para cada GeoDataFrame
crs_rindes = rindes_gdf.estimate_utm_crs()
crs_lotes = lotes_gdf.estimate_utm_crs()

print(f'CRS de los rindes: {rindes_gdf.crs}')
print(f'CRS de los lotes: {lotes_gdf.crs}')
print(f'CRS estimado de los rindes: {crs_rindes}')
print(f'CRS estimado de los lotes: {crs_lotes}')

In [ ]:
# Proyectar ambos GDFs al CRS UTM
rindes_gdf = rindes_gdf.to_crs(crs_rindes)
lotes_gdf = lotes_gdf.to_crs(crs_lotes)

print(f'CRS de los rindes: {rindes_gdf.crs}')
print(f'CRS de los lotes: {lotes_gdf.crs}')

In [ ]:
# Filtrar el lote específico
lote_especifico = lotes_gdf[lotes_gdf['Lote'] == 'SM9']

lote_especifico

In [ ]:
# Extraer puntos dentro del polígono especificado
puntos_dentro_lote = rindes_gdf[rindes_gdf.geometry.within(lote_especifico.unary_union)]

# Volver a proyectar los puntos a EPSG:4326 para visualización
puntos_dentro_lote = puntos_dentro_lote.to_crs("EPSG:4326")

print(f'CRS de los puntos dentro del lote: {puntos_dentro_lote.crs}')

# estadisticas de resumen del gdf:
# puntos_dentro_lote.describe()

In [ ]:
# Visualización con Matplotlib
fig, ax = plt.subplots(figsize=(10, 10))

puntos_dentro_lote.plot(ax=ax,
                        column='cuantiles',
                        cmap="RdYlGn",
                        markersize=2,
                        )
ax.set_axis_off()

plt.show()

In [ ]:
media_rinde = puntos_dentro_lote['Yld_Mass_D'].mean()
desviacion_estandar_rinde = puntos_dentro_lote['Yld_Mass_D'].std()

media_velocidad = puntos_dentro_lote['Speed_km_h'].mean()
desviacion_estandar_velocidad = puntos_dentro_lote['Speed_km_h'].std()

media_distancia = puntos_dentro_lote['Distance_m'].mean()
desviacion_estandar_distancia = puntos_dentro_lote['Distance_m'].std()

# Filtrado de datos a partir de la media mas menos tres desvíos standar a partir del campo Yld_Mass_D

puntos_dentro_lote_filtrados = puntos_dentro_lote[(puntos_dentro_lote['Yld_Mass_D'] > media_rinde - 3 * desviacion_estandar_rinde) &
                                                  (puntos_dentro_lote['Yld_Mass_D'] < media_rinde + 3 * desviacion_estandar_rinde) &
                                                  (puntos_dentro_lote['Speed_km_h'] > media_velocidad - 3 * desviacion_estandar_velocidad) &
                                                  (puntos_dentro_lote['Speed_km_h'] < media_velocidad + 3 * desviacion_estandar_velocidad) &
                                                  (puntos_dentro_lote['Distance_m'] > media_distancia - 3 * desviacion_estandar_distancia) &
                                                  (puntos_dentro_lote['Distance_m'] < media_distancia + 3 * desviacion_estandar_distancia)
                                                  ]

print(f'Cantidad de puntos dentro del lote: {len(puntos_dentro_lote)}')
print(f'Cantidad de puntos dentro del lote filtrados: {len(puntos_dentro_lote_filtrados)}')

In [ ]:
# Visualización con Matplotlib
fig, ax = plt.subplots(figsize=(10, 10))

puntos_dentro_lote_filtrados.plot(ax=ax,
                        column='Yld_Mass_D',
                        cmap="RdYlGn",
                        markersize=2,
                        )
ax.set_axis_off()

plt.show()

### Bibliografia extra complementaria

* [Qiusheng Wu](https://github.com/giswqs)


# Trabajo Final:

* Extraer los datos de rinde del lote 10
* Hacer un histograma y un boxplot de los datos de rinde
* Filtrar el gdf de rinde para excluir los datos que tengan en primer lugar, valores de rinde = 0 y en segundo lugar que tengan valores superiores a la media mas tres desvíos estandar e inferiores a la media menos tres desvíos estandar para los datos de rinde, ancho de labor, distancia y velocidad
* Hacer un histograma y un boxplot de los datos de rinde ya filtrado
* Graficar el mapa de rindes ya filtrado, asignarle titulo
* Extraer el centroide del lote y a partir de esas coordenadas obtener los datos de precipitación y temperatura para todo diciembre del 2021. Graficar los datos.

Se debe entregar un colab individual al mail mbonelli95@gmail.com con copia a nicoestebansoria@gmail.com y thomasvarela1990@gmail.com antes del jueves 3 de octubre.

El jueves 26 de septiembre habrá una clase de consulta.